In [1]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
from matplotlib import image as imgg
from torch.utils.data import TensorDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import utils
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
import torchvision
from torchinfo import summary
# import os

In [2]:
le = LabelEncoder()
train_df = pd.read_csv(r"/home/sanketwr/project1/practise /hackathon data_centric/data_main/train-labels.csv")


In [3]:
le.fit(train_df['label'])
train_df['transformed_label'] = le.transform(train_df['label'])
train_df.head()

,id,img,label,transformed_label
0,0,wmzihn.png,c,9
1,1,txdcgl.png,h,14
2,2,bngxil.png,y,31
3,3,nsunix.png,p,22
4,4,yxkyuc.png,j,16


In [4]:
train_df.to_csv('/home/sanketwr/project1/practise /hackathon data_centric/data_main/train_csv.csv')

In [5]:
class mydata(TensorDataset):
    def __init__(self,path_csv,path_data,height,width,type = 'train'):
        ##
        self.type = type
        self.transforms = transforms.Compose([transforms.ToTensor()])
        self.path_csv =  path_csv
        self.path_data = path_data
        self.label_csv = pd.read_csv(self.path_csv)
        if self.type == 'train':
            self.labels = self.label_csv.iloc[:,4]
            self.img_names = self.label_csv.iloc[:,2]
        else:
            self.img_names = self.label_csv.iloc[:,1]
        self.height = height
        self.width = width
        # self.to_tensor = transforms.ToTensor()
        self.data_len = len(self.label_csv)



    def __getitem__(self,idx):
        # print(self.img_names)
        path_img = self.path_data + self.img_names[idx]
        img = imgg.imread(path_img)
        img = np.asarray(img)
        img = self.transforms(img)
        if self.type == 'train':
            img_label = self.labels[idx]
            return (img,img_label)
        return img
    
    def __len__(self):
        return self.data_len
    

In [6]:
train_csv_path = r"/home/sanketwr/project1/practise /hackathon data_centric/data_main/train_csv.csv"
train_data_path = r"/home/sanketwr/project1/practise /hackathon data_centric/data_main/train/"
test_csv_path = r"/home/sanketwr/project1/practise /hackathon data_centric/data_main/val-labels.csv"
test_data_path = r"/home/sanketwr/project1/practise /hackathon data_centric/data_main/val/"
height = 60
width = 60
train_data = mydata(path_csv=train_csv_path,path_data=train_data_path,height=height,width=width,type='train')
train_loader = DataLoader(dataset=train_data,batch_size=6,shuffle=True)
test_data = mydata(path_csv=test_csv_path,path_data=test_data_path,height=height,width=width,type='test')
test_loader = DataLoader(dataset=test_data,batch_size=1,shuffle=True)
# transformss = transforms.Compose([transforms.ToTensor()])
# for imgs,labels in enumerate(train_data):
#     print(imgs,labels[0].shape,labels[1])

In [7]:
## classification network
class NN(nn.Module):
    def __init__(self,in_channel = 1,out_classes = 33):
        super(NN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=20,kernel_size=(5,5),stride = (1,1),padding=(2,2))
        self.pool  = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=20,out_channels=20,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=20,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1 = nn.Linear(in_features=20*7*7,out_features=out_classes)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        # print(x.shape)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)

        return x

# x = torch.randn(64,1,60,60)
# print(model(x).shape)



In [8]:
device = torch.device('cuda:0')
in_channel = 1
num_classes = 33
learning_rate = 0.001
batch_size = 6
num_epochs = 30
iter = 0
model = NN().to(device=device)
obj_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        print(batch_idx,data.shape)
        targets = torch.tensor(targets)
        data = data.to(device = device)
        targets = targets.to(device = device)

        # print(batch_idx)
        score = model(data)
        loss = obj_fn(score,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


0 torch.Size([6, 1, 60, 60])


/tmp/ipykernel_140141/827858190.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets)


1 torch.Size([6, 1, 60, 60])
2 torch.Size([6, 1, 60, 60])
3 torch.Size([6, 1, 60, 60])
4 torch.Size([6, 1, 60, 60])
5 torch.Size([6, 1, 60, 60])
6 torch.Size([6, 1, 60, 60])
7 torch.Size([6, 1, 60, 60])
8 torch.Size([6, 1, 60, 60])
9 torch.Size([6, 1, 60, 60])
10 torch.Size([6, 1, 60, 60])
11 torch.Size([6, 1, 60, 60])
12 torch.Size([6, 1, 60, 60])
13 torch.Size([6, 1, 60, 60])
14 torch.Size([6, 1, 60, 60])
15 torch.Size([6, 1, 60, 60])
16 torch.Size([6, 1, 60, 60])
17 torch.Size([6, 1, 60, 60])
18 torch.Size([6, 1, 60, 60])
19 torch.Size([6, 1, 60, 60])
20 torch.Size([6, 1, 60, 60])
21 torch.Size([6, 1, 60, 60])
22 torch.Size([6, 1, 60, 60])
23 torch.Size([6, 1, 60, 60])
24 torch.Size([6, 1, 60, 60])
25 torch.Size([6, 1, 60, 60])
26 torch.Size([6, 1, 60, 60])
27 torch.Size([6, 1, 60, 60])
28 torch.Size([6, 1, 60, 60])
29 torch.Size([6, 1, 60, 60])
30 torch.Size([6, 1, 60, 60])
31 torch.Size([6, 1, 60, 60])
32 torch.Size([6, 1, 60, 60])
33 torch.Size([6, 1, 60, 60])
34 torch.Size([6, 1

In [9]:
num_corr = 0
num_samples = 0
model.eval()
with torch.no_grad():
    for x,y in train_loader:
        x = x.to(device=device)
        y = y.to(device = device)
        # x =x.reshape(x.shape[0],-1)
        
        scores = model(x)
        # shape of score : 64x10 --you want the max of this 10 column entries as the predicted class
        _,predictions = scores.max(1) # max along axis 1 or columns and returns val,index we need just the index
        num_corr = num_corr + (predictions == y).sum()
        num_samples = num_samples + predictions.size(0)
print(f'got {num_corr}/{num_samples} with accuracy {float(num_corr)/float(num_samples)*100:.2f}')


got 8778/9402 with accuracy 93.36


In [24]:
weights = torchvision.models.ResNet18_Weights.DEFAULT
model_resnet = torchvision.models.resnet18(weights=weights).to(device=device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/sanketwr/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [25]:
weight_conv1=model_resnet.conv1.weight.clone()
model_resnet.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7, stride=2, padding=3, bias=False)
with torch.no_grad():
    model_resnet.conv1.weight[:,0,:,:] = torch.nn.Parameter(weight_conv1[:,0,:,:])
    # model_resnet.conv1.weight[:,:,:,:] = torch.nn.Parameter(weight_conv1[:,1,:,:])
for param in model_resnet.parameters():
    param.requires_grad = False
model_resnet.fc = nn.Linear(in_features=model_resnet.fc.in_features,out_features= num_classes)

In [26]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model_resnet, 
        input_size=(6, 1, 60, 60), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [6, 1, 60, 60]       [6, 33]              --                   Partial
├─Conv2d (conv1)                         [6, 1, 60, 60]       [6, 64, 30, 30]      (3,136)              False
├─BatchNorm2d (bn1)                      [6, 64, 30, 30]      [6, 64, 30, 30]      (128)                False
├─ReLU (relu)                            [6, 64, 30, 30]      [6, 64, 30, 30]      --                   --
├─MaxPool2d (maxpool)                    [6, 64, 30, 30]      [6, 64, 15, 15]      --                   --
├─Sequential (layer1)                    [6, 64, 15, 15]      [6, 64, 15, 15]      --                   False
│    └─BasicBlock (0)                    [6, 64, 15, 15]      [6, 64, 15, 15]      --                   False
│    │    └─Conv2d (conv1)               [6, 64, 15, 15]      [6, 64, 15, 15]      (36,864)             False
│    │    

In [27]:

learning_rate = 0.001
obj_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(),lr = learning_rate)
train_loss = []
train_acc = []
total_step = len(train_loader)

for epoch in range(1,num_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx,(data,targets) in enumerate(train_loader):
        # print(batch_idx,data.shape)
        targets = torch.tensor(targets)
        data = data.to(device = device)
        targets = targets.to(device = device)

        # print(batch_idx)
        score = model_resnet(data)
        loss = obj_fn(score,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = score.max(1)
        correct += torch.sum(pred==targets).item()
        total += targets.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, num_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')

Epoch 1

Epoch [1/30], Step [0/1567], Loss: 4.0034


/tmp/ipykernel_140141/3420118933.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets)


Epoch [1/30], Step [20/1567], Loss: 4.5176
Epoch [1/30], Step [40/1567], Loss: 3.8268
Epoch [1/30], Step [60/1567], Loss: 4.0550
Epoch [1/30], Step [80/1567], Loss: 3.5268
Epoch [1/30], Step [100/1567], Loss: 3.5571
Epoch [1/30], Step [120/1567], Loss: 3.2225
Epoch [1/30], Step [140/1567], Loss: 3.6824
Epoch [1/30], Step [160/1567], Loss: 4.6589
Epoch [1/30], Step [180/1567], Loss: 3.8523
Epoch [1/30], Step [200/1567], Loss: 2.9796
Epoch [1/30], Step [220/1567], Loss: 3.4461
Epoch [1/30], Step [240/1567], Loss: 2.7066
Epoch [1/30], Step [260/1567], Loss: 3.0320
Epoch [1/30], Step [280/1567], Loss: 2.9695
Epoch [1/30], Step [300/1567], Loss: 3.9121
Epoch [1/30], Step [320/1567], Loss: 3.9903
Epoch [1/30], Step [340/1567], Loss: 3.0334
Epoch [1/30], Step [360/1567], Loss: 2.8927
Epoch [1/30], Step [380/1567], Loss: 3.7661
Epoch [1/30], Step [400/1567], Loss: 3.4209
Epoch [1/30], Step [420/1567], Loss: 2.8835
Epoch [1/30], Step [440/1567], Loss: 3.3162
Epoch [1/30], Step [460/1567], Loss:

KeyboardInterrupt: 

In [24]:
preds = []
for x in test_loader:
    # print(1)
    x = x.to(device = device)
    scores = model(x)
    _,predictions = scores.max(1)
    preds.append(predictions.cpu().numpy())

In [26]:
val_labels = le.inverse_transform(preds)

/home/sanketwr/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
val_labels.shape

(1109,)

In [30]:
df = pd.DataFrame(data = val_labels,columns=['label'])
df.head

<bound method NDFrame.head of      label
0        n
1        v
2        j
3        w
4        q
...    ...
1104     p
1105     f
1106     2
1107     g
1108     q

[1109 rows x 1 columns]>

In [32]:
df.to_csv('/home/sanketwr/project1/practise /hackathon data_centric/data_main/test_labels.csv')